In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/protein-localization/sample_submission.csv
/kaggle/input/protein-localization/field_descriptions.txt
/kaggle/input/protein-localization/protein_interactions.csv
/kaggle/input/protein-localization/train.csv
/kaggle/input/protein-localization/test.csv
/kaggle/input/protein-localization/label_legend.csv


## DATA PRE-PROCESSING

In [2]:
train_df=pd.read_csv('../input/protein-localization/train.csv',header=None)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (444) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import io
labels=[]
file=open("../input/protein-localization/field_descriptions.txt",'r')
for i in file:
    labels.append(i.split("\t")[0].strip(":"))

In [4]:
train_df.columns=labels
train_df.head()

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,FUNCTION ENERGY,FUNCTION IONIC HOMEOSTASIS,FUNCTION METABOLISM,FUNCTION PROTEIN DESTINATION,FUNCTION PROTEIN SYNTHESIS,FUNCTION TRANSCRIPTION,FUNCTION TRANSPORT FACILITATION,FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS,LOCALIZATION,Label
0,P239476,Non-Essential,No,No,No,No,No,No,No,No,...,No,No,Yes,No,No,Yes,No,No,nucleus,0
1,P234427,Non-Essential,No,No,No,No,No,No,No,No,...,No,No,No,No,No,Yes,No,No,nucleus,0
2,P234429,Non-Essential,No,No,No,Yes,No,No,No,No,...,No,No,No,No,No,No,No,No,nucleus,0
3,P234430,Non-Essential,No,No,No,No,No,No,No,No,...,No,Yes,No,No,No,Yes,No,No,nucleus,0
4,P239467,Essential,No,No,No,No,No,No,No,No,...,No,No,No,No,No,Yes,No,No,nucleus,0


In [5]:
len(train_df[train_df['ESSENTIAL']=="?"])

44

In [6]:
# cols that have only "?" values
cols_train=[]
for co in train_df.columns:
        if (set(train_df[co].values)==set(['?'])):
            cols_train.append(co)

In [7]:
train_df.drop(cols_train,axis=1,inplace=True)

In [8]:
train_df.head()

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,FUNCTION ENERGY,FUNCTION IONIC HOMEOSTASIS,FUNCTION METABOLISM,FUNCTION PROTEIN DESTINATION,FUNCTION PROTEIN SYNTHESIS,FUNCTION TRANSCRIPTION,FUNCTION TRANSPORT FACILITATION,FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS,LOCALIZATION,Label
0,P239476,Non-Essential,No,No,No,No,No,No,No,No,...,No,No,Yes,No,No,Yes,No,No,nucleus,0
1,P234427,Non-Essential,No,No,No,No,No,No,No,No,...,No,No,No,No,No,Yes,No,No,nucleus,0
2,P234429,Non-Essential,No,No,No,Yes,No,No,No,No,...,No,No,No,No,No,No,No,No,nucleus,0
3,P234430,Non-Essential,No,No,No,No,No,No,No,No,...,No,Yes,No,No,No,Yes,No,No,nucleus,0
4,P239467,Essential,No,No,No,No,No,No,No,No,...,No,No,No,No,No,Yes,No,No,nucleus,0


In [9]:
for i in range(len(train_df)):
    if train_df.loc[i,'ESSENTIAL']=='?':
        train_df.loc[i,'ESSENTIAL']=train_df['ESSENTIAL'].mode()[0]
train_df['ESSENTIAL']

0      Non-Essential
1      Non-Essential
2      Non-Essential
3      Non-Essential
4          Essential
           ...      
857    Non-Essential
858    Non-Essential
859    Non-Essential
860        Essential
861        Essential
Name: ESSENTIAL, Length: 862, dtype: object

In [10]:
len(train_df[train_df['ESSENTIAL']=="?"])

0

In [11]:
train_df

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,FUNCTION ENERGY,FUNCTION IONIC HOMEOSTASIS,FUNCTION METABOLISM,FUNCTION PROTEIN DESTINATION,FUNCTION PROTEIN SYNTHESIS,FUNCTION TRANSCRIPTION,FUNCTION TRANSPORT FACILITATION,FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS,LOCALIZATION,Label
0,P239476,Non-Essential,No,No,No,No,No,No,No,No,...,No,No,Yes,No,No,Yes,No,No,nucleus,0
1,P234427,Non-Essential,No,No,No,No,No,No,No,No,...,No,No,No,No,No,Yes,No,No,nucleus,0
2,P234429,Non-Essential,No,No,No,Yes,No,No,No,No,...,No,No,No,No,No,No,No,No,nucleus,0
3,P234430,Non-Essential,No,No,No,No,No,No,No,No,...,No,Yes,No,No,No,Yes,No,No,nucleus,0
4,P239467,Essential,No,No,No,No,No,No,No,No,...,No,No,No,No,No,Yes,No,No,nucleus,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,P235892,Non-Essential,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,cytoskeleton,3
858,P235906,Non-Essential,No,No,No,No,No,No,No,No,...,No,No,No,Yes,No,No,No,No,transport vesicles,8
859,P235897,Non-Essential,No,No,No,No,No,No,No,No,...,No,No,No,No,Yes,No,No,No,cytoplasm,1
860,P235905,Essential,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,cytoskeleton,3


In [12]:
train_labels = train_df['Label']
train_df.drop(['Label'],axis=1,inplace=True)
train_df.drop(['LOCALIZATION'],axis=1,inplace=True)

In [13]:
train_df

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle),FUNCTION CELLULAR TRANSPORT AND TRANSPORTMECHANISMS,FUNCTION ENERGY,FUNCTION IONIC HOMEOSTASIS,FUNCTION METABOLISM,FUNCTION PROTEIN DESTINATION,FUNCTION PROTEIN SYNTHESIS,FUNCTION TRANSCRIPTION,FUNCTION TRANSPORT FACILITATION,FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS
0,P239476,Non-Essential,No,No,No,No,No,No,No,No,...,Yes,No,No,No,Yes,No,No,Yes,No,No
1,P234427,Non-Essential,No,No,No,No,No,No,No,No,...,Yes,No,No,No,No,No,No,Yes,No,No
2,P234429,Non-Essential,No,No,No,Yes,No,No,No,No,...,Yes,No,No,No,No,No,No,No,No,No
3,P234430,Non-Essential,No,No,No,No,No,No,No,No,...,Yes,No,No,Yes,No,No,No,Yes,No,No
4,P239467,Essential,No,No,No,No,No,No,No,No,...,Yes,No,No,No,No,No,No,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,P235892,Non-Essential,No,No,No,No,No,No,No,No,...,Yes,Yes,No,No,No,No,No,No,No,No
858,P235906,Non-Essential,No,No,No,No,No,No,No,No,...,Yes,Yes,No,No,No,Yes,No,No,No,No
859,P235897,Non-Essential,No,No,No,No,No,No,No,No,...,Yes,No,No,No,No,No,Yes,No,No,No
860,P235905,Essential,No,No,No,No,No,No,No,No,...,Yes,No,No,No,No,No,No,No,No,No


In [14]:
train_labels

0      0
1      0
2      0
3      0
4      0
      ..
857    3
858    8
859    1
860    3
861    1
Name: Label, Length: 862, dtype: int64

In [15]:
test_df=pd.read_csv('../input/protein-localization/test.csv',header=None)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (524,720,768,1112,1182,1288,1302,1352,1354,1378,1434,1436,1488,1502,1504,1604,1608,1734,1838,1908,1914,1942,1996,2246,2270,2328,2460,2514,2576,2620,2724,2758,2930) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,2950,2951,2952,2953,2954,2955,2956,2957,2958,2959
0,P234273,Essential,No,No,No,Yes,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
1,P234490,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
2,P236380,?,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
3,P234394,Essential,No,Yes,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
4,P234385,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,P235517,Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
377,P239167,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
378,P240327,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
379,P235928,Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?


In [17]:
labels2 = [] 
labels2 = labels
labels2.remove("Label")
len(labels2)

2960

In [18]:
test_df.columns=labels2
test_df.head()

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,FUNCTION CELLULAR TRANSPORT AND TRANSPORTMECHANISMS,FUNCTION ENERGY,FUNCTION IONIC HOMEOSTASIS,FUNCTION METABOLISM,FUNCTION PROTEIN DESTINATION,FUNCTION PROTEIN SYNTHESIS,FUNCTION TRANSCRIPTION,FUNCTION TRANSPORT FACILITATION,FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS,LOCALIZATION
0,P234273,Essential,No,No,No,Yes,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
1,P234490,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
2,P236380,?,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
3,P234394,Essential,No,Yes,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
4,P234385,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?


In [19]:
test_df = test_df.iloc[: , :-1]

In [20]:
test_df

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle),FUNCTION CELLULAR TRANSPORT AND TRANSPORTMECHANISMS,FUNCTION ENERGY,FUNCTION IONIC HOMEOSTASIS,FUNCTION METABOLISM,FUNCTION PROTEIN DESTINATION,FUNCTION PROTEIN SYNTHESIS,FUNCTION TRANSCRIPTION,FUNCTION TRANSPORT FACILITATION,FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS
0,P234273,Essential,No,No,No,Yes,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
1,P234490,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
2,P236380,?,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
3,P234394,Essential,No,Yes,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
4,P234385,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,P235517,Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
377,P239167,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
378,P240327,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
379,P235928,Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?


In [21]:
test_df.drop(cols_train,axis=1,inplace=True)

In [22]:
test_df

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle),FUNCTION CELLULAR TRANSPORT AND TRANSPORTMECHANISMS,FUNCTION ENERGY,FUNCTION IONIC HOMEOSTASIS,FUNCTION METABOLISM,FUNCTION PROTEIN DESTINATION,FUNCTION PROTEIN SYNTHESIS,FUNCTION TRANSCRIPTION,FUNCTION TRANSPORT FACILITATION,FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS
0,P234273,Essential,No,No,No,Yes,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
1,P234490,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
2,P236380,?,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
3,P234394,Essential,No,Yes,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
4,P234385,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,P235517,Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
377,P239167,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
378,P240327,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
379,P235928,Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?


In [23]:
#to remove all rows in essential that have "?" in test
for i in range(len(test_df)):
    if test_df.loc[i,'ESSENTIAL']=='?':
        test_df.loc[i,'ESSENTIAL']=test_df['ESSENTIAL'].mode()[0]
test_df['ESSENTIAL']

0          Essential
1      Non-Essential
2      Non-Essential
3          Essential
4      Non-Essential
           ...      
376        Essential
377    Non-Essential
378    Non-Essential
379        Essential
380    Non-Essential
Name: ESSENTIAL, Length: 381, dtype: object

In [24]:
len(test_df[test_df['ESSENTIAL']=="?"])

0

In [25]:
test_df

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle),FUNCTION CELLULAR TRANSPORT AND TRANSPORTMECHANISMS,FUNCTION ENERGY,FUNCTION IONIC HOMEOSTASIS,FUNCTION METABOLISM,FUNCTION PROTEIN DESTINATION,FUNCTION PROTEIN SYNTHESIS,FUNCTION TRANSCRIPTION,FUNCTION TRANSPORT FACILITATION,FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS
0,P234273,Essential,No,No,No,Yes,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
1,P234490,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
2,P236380,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
3,P234394,Essential,No,Yes,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
4,P234385,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,P235517,Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
377,P239167,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
378,P240327,Non-Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?
379,P235928,Essential,No,No,No,No,No,No,No,No,...,?,?,?,?,?,?,?,?,?,?


In [26]:
#to remove all columns that have "?" in test
cols_test=[]
for co in test_df.columns:
        if (set(test_df[co].values)==set(['?'])):
            cols_test.append(co)

In [27]:
test_df.drop(cols_test,axis=1,inplace=True)

In [28]:
test_df

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,INTERACTING PROTEIN P235892 Type,INTERACTING PROTEIN P235892 Corr,INTERACTING PROTEIN P235906 Type,INTERACTING PROTEIN P235906 Corr,INTERACTING PROTEIN P235897 Type,INTERACTING PROTEIN P235897 Corr,INTERACTING PROTEIN P235905 Type,INTERACTING PROTEIN P235905 Corr,INTERACTING PROTEIN P235899 Type,INTERACTING PROTEIN P235899 Corr
0,P234273,Essential,No,No,No,Yes,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0
1,P234490,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0
2,P236380,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0
3,P234394,Essential,No,Yes,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0
4,P234385,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,P235517,Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0
377,P239167,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0
378,P240327,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0
379,P235928,Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0


In [29]:
# removing the same columns from test in train as well
train_df.drop(cols_test,axis=1,inplace=True)

In [30]:
train_df

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,INTERACTING PROTEIN P235892 Type,INTERACTING PROTEIN P235892 Corr,INTERACTING PROTEIN P235906 Type,INTERACTING PROTEIN P235906 Corr,INTERACTING PROTEIN P235897 Type,INTERACTING PROTEIN P235897 Corr,INTERACTING PROTEIN P235905 Type,INTERACTING PROTEIN P235905 Corr,INTERACTING PROTEIN P235899 Type,INTERACTING PROTEIN P235899 Corr
0,P239476,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0,No,0,No,0,No,0.0
1,P234427,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0,No,0,No,0,No,0.0
2,P234429,Non-Essential,No,No,No,Yes,No,No,No,No,...,No,0.0,No,0,No,0,No,0,No,0.0
3,P234430,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0,No,0,No,0,No,0.0
4,P239467,Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0,No,0,No,0,No,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,P235892,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0,No,0,No,0,No,0.0
858,P235906,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0,No,0,No,0,No,0.0
859,P235897,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0,No,0,No,0,No,0.0
860,P235905,Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0,No,0,No,0,No,0.0


In [31]:
# combining train and test to do one hot encoding since string columns are present
df_new = pd.concat([train_df,test_df],ignore_index=True)
df_new

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS Dehydrogenases,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),...,INTERACTING PROTEIN P235892 Type,INTERACTING PROTEIN P235892 Corr,INTERACTING PROTEIN P235906 Type,INTERACTING PROTEIN P235906 Corr,INTERACTING PROTEIN P235897 Type,INTERACTING PROTEIN P235897 Corr,INTERACTING PROTEIN P235905 Type,INTERACTING PROTEIN P235905 Corr,INTERACTING PROTEIN P235899 Type,INTERACTING PROTEIN P235899 Corr
0,P239476,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0.0
1,P234427,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0.0
2,P234429,Non-Essential,No,No,No,Yes,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0.0
3,P234430,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0.0
4,P239467,Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,P235517,Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0.0
1239,P239167,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0.0
1240,P240327,Non-Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0.0
1241,P235928,Essential,No,No,No,No,No,No,No,No,...,No,0.0,No,0.0,No,0.0,No,0,No,0.0


In [32]:
# removing columns based on correlation 
cols_to_remove = []
df_new_corr = df_new.corr()

In [33]:
df_new_corr

,"""NUM INTERACTING WITH FUNCTION """"CELL GROWTH""","""NUM INTERACTING WITH FUNCTION """"CELL RESCUE""",NUM INTERACTING WITH FUNCTION CELLULAR BIOGENESIS (proteins are not localized to the corresponding organelle),NUM INTERACTING WITH FUNCTION CELLULAR COMMUNICATION/SIGNAL TRANSDUCTION,NUM INTERACTING WITH FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle),NUM INTERACTING WITH FUNCTION CELLULAR TRANSPORT AND TRANSPORTMECHANISMS,NUM INTERACTING WITH FUNCTION ENERGY,NUM INTERACTING WITH FUNCTION IONIC HOMEOSTASIS,NUM INTERACTING WITH FUNCTION METABOLISM,NUM INTERACTING WITH FUNCTION PROTEIN DESTINATION,...,INTERACTING PROTEIN P235881 Corr,INTERACTING PROTEIN P235914 Corr,INTERACTING PROTEIN P235885 Corr,INTERACTING PROTEIN P235912 Corr,INTERACTING PROTEIN P235889 Corr,INTERACTING PROTEIN P235892 Corr,INTERACTING PROTEIN P235906 Corr,INTERACTING PROTEIN P235897 Corr,INTERACTING PROTEIN P235905 Corr,INTERACTING PROTEIN P235899 Corr
"""NUM INTERACTING WITH FUNCTION """"CELL GROWTH""",1.000000,0.277985,0.283160,0.412342,0.637581,0.178774,0.115610,0.092798,0.290726,0.216955,...,-0.022366,NaN,-0.023457,0.058051,NaN,0.026536,-0.015851,-0.015851,NaN,-0.026295
"""NUM INTERACTING WITH FUNCTION """"CELL RESCUE""",0.277985,1.000000,0.013546,0.307590,0.201990,0.011248,0.166292,0.108061,0.178532,0.111321,...,-0.012222,NaN,-0.026341,-0.015047,NaN,-0.018869,-0.008662,-0.008662,NaN,0.008662
NUM INTERACTING WITH FUNCTION CELLULAR BIOGENESIS (proteins are not localized to the corresponding organelle),0.283160,0.013546,1.000000,0.067905,0.274275,0.155825,0.033424,0.022213,0.048444,0.417408,...,-0.011991,NaN,0.047906,0.000889,NaN,0.027861,0.058357,-0.008498,NaN,-0.058357
NUM INTERACTING WITH FUNCTION CELLULAR COMMUNICATION/SIGNAL TRANSDUCTION,0.412342,0.307590,0.067905,1.000000,0.240472,0.013797,0.060018,0.171329,0.210906,0.006923,...,-0.009987,NaN,-0.005811,0.018772,NaN,0.002750,-0.007078,-0.007078,NaN,-0.057616
NUM INTERACTING WITH FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle),0.637581,0.201990,0.274275,0.240472,1.000000,0.509568,0.177669,0.089085,0.427271,0.381227,...,0.007785,NaN,-0.022211,0.076936,NaN,0.000566,-0.011732,-0.011732,NaN,-0.026146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INTERACTING PROTEIN P235892 Corr,0.026536,-0.018869,0.027861,0.002750,0.000566,0.033800,-0.019079,-0.007172,-0.021086,-0.023098,...,-0.001666,NaN,-0.000969,-0.002051,NaN,1.000000,-0.001181,-0.001181,NaN,0.001181
INTERACTING PROTEIN P235906 Corr,-0.015851,-0.008662,0.058357,-0.007078,-0.011732,0.010039,-0.005745,-0.004890,-0.011429,0.022961,...,-0.001136,NaN,-0.000661,-0.001399,NaN,-0.001181,1.000000,-0.000805,NaN,0.000805
INTERACTING PROTEIN P235897 Corr,-0.015851,-0.008662,-0.008498,-0.007078,-0.011732,-0.011291,-0.005745,-0.004890,-0.011429,-0.012939,...,-0.001136,NaN,-0.000661,-0.001399,NaN,-0.001181,-0.000805,1.000000,NaN,0.000805
INTERACTING PROTEIN P235905 Corr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
cols_to_remove=set()
for j in df_new_corr.columns:
        if df_new_corr.loc[j,j]>0.6:
            cols_to_remove.add(j)

len(cols_to_remove)

751

In [35]:
for co in df_new.columns:
        if set(df_new[co].values)== set(['No']) or set(df_new[co].values)== set(['Yes']):
            cols_to_remove.add(co)

In [36]:
df_new.drop(cols_to_remove,axis=1,inplace=True)

In [37]:
df_new

,Protein,ESSENTIAL,CLASS Actin related proteins,CLASS Actins,CLASS Adaptins,CLASS ATPases,CLASS Cyclins,CLASS GTP-binding proteins,CLASS GTP/GDP dissociation inhibitors (GDIs),CLASS GTP/GDP-exchange factors (GEFs),...,INTERACTING PROTEIN P235881 Type,INTERACTING PROTEIN P235914 Corr,INTERACTING PROTEIN P235885 Type,INTERACTING PROTEIN P235912 Type,INTERACTING PROTEIN P235889 Corr,INTERACTING PROTEIN P235892 Type,INTERACTING PROTEIN P235906 Type,INTERACTING PROTEIN P235897 Type,INTERACTING PROTEIN P235905 Corr,INTERACTING PROTEIN P235899 Type
0,P239476,Non-Essential,No,No,No,No,No,No,No,No,...,No,0,No,No,0,No,No,No,0,No
1,P234427,Non-Essential,No,No,No,No,No,No,No,No,...,No,0,No,No,0,No,No,No,0,No
2,P234429,Non-Essential,No,No,No,Yes,No,No,No,No,...,No,0,No,No,0,No,No,No,0,No
3,P234430,Non-Essential,No,No,No,No,No,No,No,No,...,No,0,No,No,0,No,No,No,0,No
4,P239467,Essential,No,No,No,No,No,No,No,No,...,No,0,No,No,0,No,No,No,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,P235517,Essential,No,No,No,No,No,No,No,No,...,No,0,No,No,0,No,No,No,0,No
1239,P239167,Non-Essential,No,No,No,No,No,No,No,No,...,No,0,No,No,0,No,No,No,0,No
1240,P240327,Non-Essential,No,No,No,No,No,No,No,No,...,No,0,No,No,0,No,No,No,0,No
1241,P235928,Essential,No,No,No,No,No,No,No,No,...,No,0,No,No,0,No,No,No,0,No


In [38]:
# one hot encoding
df_new=df_new.iloc[:,1:]
df_new=pd.get_dummies(df_new)
df_new

,"""NUM INTERACTING WITH FUNCTION """"TRANSPOSABLE ELEMENTS""",INTERACTING PROTEIN P239476 Corr,INTERACTING PROTEIN P234437 Corr,INTERACTING PROTEIN P239438 Corr,INTERACTING PROTEIN P234452 Corr,INTERACTING PROTEIN P234509 Corr,INTERACTING PROTEIN P239294 Corr,INTERACTING PROTEIN P239196 Corr,INTERACTING PROTEIN P239186 Corr,INTERACTING PROTEIN P234630 Corr,...,INTERACTING PROTEIN P235912 Type_No,INTERACTING PROTEIN P235892 Type_Genetic,INTERACTING PROTEIN P235892 Type_No,INTERACTING PROTEIN P235892 Type_Physical,INTERACTING PROTEIN P235906 Type_No,INTERACTING PROTEIN P235906 Type_Physical,INTERACTING PROTEIN P235897 Type_Genetic,INTERACTING PROTEIN P235897 Type_No,INTERACTING PROTEIN P235899 Type_Genetic,INTERACTING PROTEIN P235899 Type_No
0,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
2,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
3,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
4,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1239,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1240,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1241,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1


In [39]:
np.any(np.isnan(df_new))

True

In [40]:
df_new = df_new.replace(np.nan, 0)
df_new

,"""NUM INTERACTING WITH FUNCTION """"TRANSPOSABLE ELEMENTS""",INTERACTING PROTEIN P239476 Corr,INTERACTING PROTEIN P234437 Corr,INTERACTING PROTEIN P239438 Corr,INTERACTING PROTEIN P234452 Corr,INTERACTING PROTEIN P234509 Corr,INTERACTING PROTEIN P239294 Corr,INTERACTING PROTEIN P239196 Corr,INTERACTING PROTEIN P239186 Corr,INTERACTING PROTEIN P234630 Corr,...,INTERACTING PROTEIN P235912 Type_No,INTERACTING PROTEIN P235892 Type_Genetic,INTERACTING PROTEIN P235892 Type_No,INTERACTING PROTEIN P235892 Type_Physical,INTERACTING PROTEIN P235906 Type_No,INTERACTING PROTEIN P235906 Type_Physical,INTERACTING PROTEIN P235897 Type_Genetic,INTERACTING PROTEIN P235897 Type_No,INTERACTING PROTEIN P235899 Type_Genetic,INTERACTING PROTEIN P235899 Type_No
0,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
2,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
3,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
4,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1239,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1240,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1241,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1


In [41]:
np.any(np.isnan(df_new))

False

In [42]:
# getting back the original train_df but now with one hot encoding
X_train = df_new.iloc[:len(train_df),:]
X_train

,"""NUM INTERACTING WITH FUNCTION """"TRANSPOSABLE ELEMENTS""",INTERACTING PROTEIN P239476 Corr,INTERACTING PROTEIN P234437 Corr,INTERACTING PROTEIN P239438 Corr,INTERACTING PROTEIN P234452 Corr,INTERACTING PROTEIN P234509 Corr,INTERACTING PROTEIN P239294 Corr,INTERACTING PROTEIN P239196 Corr,INTERACTING PROTEIN P239186 Corr,INTERACTING PROTEIN P234630 Corr,...,INTERACTING PROTEIN P235912 Type_No,INTERACTING PROTEIN P235892 Type_Genetic,INTERACTING PROTEIN P235892 Type_No,INTERACTING PROTEIN P235892 Type_Physical,INTERACTING PROTEIN P235906 Type_No,INTERACTING PROTEIN P235906 Type_Physical,INTERACTING PROTEIN P235897 Type_Genetic,INTERACTING PROTEIN P235897 Type_No,INTERACTING PROTEIN P235899 Type_Genetic,INTERACTING PROTEIN P235899 Type_No
0,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
2,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
3,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
4,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
858,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
859,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
860,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1


In [43]:
# getting back the original test_df but now with one hot encoding
X_test = df_new.iloc[len(train_df):,:]
X_test

,"""NUM INTERACTING WITH FUNCTION """"TRANSPOSABLE ELEMENTS""",INTERACTING PROTEIN P239476 Corr,INTERACTING PROTEIN P234437 Corr,INTERACTING PROTEIN P239438 Corr,INTERACTING PROTEIN P234452 Corr,INTERACTING PROTEIN P234509 Corr,INTERACTING PROTEIN P239294 Corr,INTERACTING PROTEIN P239196 Corr,INTERACTING PROTEIN P239186 Corr,INTERACTING PROTEIN P234630 Corr,...,INTERACTING PROTEIN P235912 Type_No,INTERACTING PROTEIN P235892 Type_Genetic,INTERACTING PROTEIN P235892 Type_No,INTERACTING PROTEIN P235892 Type_Physical,INTERACTING PROTEIN P235906 Type_No,INTERACTING PROTEIN P235906 Type_Physical,INTERACTING PROTEIN P235897 Type_Genetic,INTERACTING PROTEIN P235897 Type_No,INTERACTING PROTEIN P235899 Type_Genetic,INTERACTING PROTEIN P235899 Type_No
862,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
863,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
864,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
865,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
866,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1239,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1240,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
1241,0,0,0,0,0,0,0.0,0,0,0,...,1,0,1,0,1,0,0,1,0,1


In [44]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862 entries, 0 to 861
Columns: 2878 entries, "NUM INTERACTING WITH FUNCTION ""TRANSPOSABLE ELEMENTS" to INTERACTING PROTEIN P235899 Type_No
dtypes: float64(11), int64(91), uint8(2776)
memory usage: 3.0 MB


In [45]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 862 to 1242
Columns: 2878 entries, "NUM INTERACTING WITH FUNCTION ""TRANSPOSABLE ELEMENTS" to INTERACTING PROTEIN P235899 Type_No
dtypes: float64(11), int64(91), uint8(2776)
memory usage: 1.3 MB


## Using standard Scalar here to have mean 0 and sd=1

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_train

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991]])

In [47]:
X_test=scaler.fit_transform(X_test)
X_test

array([[0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ]])

In [48]:
train_labels

0      0
1      0
2      0
3      0
4      0
      ..
857    3
858    8
859    1
860    3
861    1
Name: Label, Length: 862, dtype: int64

In [49]:
labels_df=pd.read_csv('../input/protein-localization/label_legend.csv')

## Random Forest HyperTuning

In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10,20,30,40,80,100],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [5, 10,15,20],
    'n_estimators': [200, 300, 400,800]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
gird_results=grid_search.fit(X_train,train_labels)

Fitting 3 folds for each of 576 candidates, totalling 1728 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


In [51]:
gird_results.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 300}

In [52]:
# rf_results.cv_results_

In [53]:
len(train_labels)

862

In [54]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
sss = KFold(n_splits=10,shuffle=True,random_state=0)

epochs=0
for train_index, val_index in sss.split(X_train, train_labels):
    #print("TRAIN:", train_index, "Val:", val_index)
    X_train_, X_val = X_train[train_index], X_train[val_index]
    y_train_, y_val = train_labels[train_index], train_labels[val_index]
    rf.fit(X_train_,y_train_)
    y_predicted=rf.predict(X_val)
    print("Epochs #: {0} -----> val accuracy : {1}".format(epochs,accuracy_score(y_predicted,y_val)))
    epochs+=1

Epochs #: 0 -----> val accuracy : 0.6781609195402298
Epochs #: 1 -----> val accuracy : 0.7011494252873564
Epochs #: 2 -----> val accuracy : 0.686046511627907
Epochs #: 3 -----> val accuracy : 0.6046511627906976
Epochs #: 4 -----> val accuracy : 0.7093023255813954
Epochs #: 5 -----> val accuracy : 0.7093023255813954
Epochs #: 6 -----> val accuracy : 0.7209302325581395
Epochs #: 7 -----> val accuracy : 0.6976744186046512
Epochs #: 8 -----> val accuracy : 0.7209302325581395
Epochs #: 9 -----> val accuracy : 0.686046511627907


In [55]:
ans=rf.predict(X_test)

In [56]:
ans

array([ 1,  0,  0,  3,  1,  0,  0,  0,  5, 10,  0,  1,  3,  0,  0,  0, 12,
        0,  0,  0,  0,  0,  0,  0,  7,  0,  1,  0,  1,  0,  0,  0,  0,  0,
        0,  1,  2,  7,  7,  4,  0,  0,  0,  0,  0,  0,  1,  7,  7,  0,  7,
        0,  5,  0,  0,  0,  1,  1,  4,  7,  0,  0,  0,  3,  1,  1,  0,  5,
        0,  1,  0,  5,  0,  8,  0,  3,  1,  1,  0,  0,  3,  0,  1,  0,  0,
        0,  0,  0,  3,  2,  0,  0,  5,  0,  1,  3,  0,  3,  3,  1,  0,  1,
        8,  0,  3,  4,  4,  0,  0,  1,  6,  0,  3,  1,  0,  0,  1,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  1,  0,  7,  0,  1,  1,  3,
        0,  0,  1,  1,  0,  8,  0,  0,  2,  8,  0,  3,  0,  0,  1,  1,  0,
        0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  3,  0,
        3,  0, 12,  3,  6,  0,  1,  8,  0,  0,  3,  6,  2,  1,  0,  0,  0,
        1,  0,  1,  0,  0,  0,  1,  4,  3,  1,  0,  7,  2,  0,  0,  2,  0,
        2,  0,  0,  1,  0,  1,  0,  3,  0,  0,  8,  0,  5,  0,  0,  0,  1,
        7,  0,  1,  8,  7

In [57]:
ans_file_rf=pd.DataFrame({'Key':test_df.iloc[:,0],'Label':ans})

In [58]:
ans_file_rf

,Key,Label
0,P234273,1
1,P234490,0
2,P236380,0
3,P234394,3
4,P234385,1
...,...,...
376,P235517,3
377,P239167,1
378,P240327,0
379,P235928,0


In [59]:
ans_file_rf.to_csv('/kaggle/working/submissions.csv',index=False)